# Import libraries

In [1]:
import os
import pandas as pd
import tweepy
import json

import urllib.request
import random 
import numpy as np
import datetime
from datetime import date
import warnings
import requests
import ipynb 

# load json with country names and codes
with urllib.request.urlopen("https://raw.githubusercontent.com/dquintani/GreenhouseData/master/supplemental/STANDARD_COUNTRY_DICT_ISO3.json") as url:
    STANDARD_COUNTRY_DICT = json.loads(url.read().decode())
    
pd.set_option('display.max_colwidth', None)

In [2]:
# environmental secrets when working locally
from dotenv import load_dotenv
try:
    load_dotenv()
    # load_dotenv("../../../../")
    print("envs loaded")
except:
    print("envs failed to load")
    pass

envs loaded


# Set up tweepy

In [3]:
#USING GITHUB ENVS

#test
GHD_TWITTER_APIKEY= os.environ["GHD_TWITTER_APIKEY"]
GHD_TWITTER_APIKEYSECRET = os.environ["GHD_TWITTER_APIKEYSECRET"]

GHD_TWITTER_BEARERTOKEN = os.environ["GHD_TWITTER_BEARERTOKEN"]
GHD_TWITTER_ACCESSTOKEN = os.environ["GHD_TWITTER_ACCESSTOKEN"]
GHD_TWITTER_ACCESSTOKENSECRET = os.environ["GHD_TWITTER_ACCESSTOKENSECRET"]
GHD_TWITTER_CLIENTID = os.environ["GHD_TWITTER_CLIENTID"]
GHD_TWITTER_CLIENTSECRET = os.environ["GHD_TWITTER_CLIENTSECRET"]

# Authenticate to Twitter
auth = tweepy.OAuthHandler(GHD_TWITTER_APIKEY, GHD_TWITTER_APIKEYSECRET)
auth.set_access_token(GHD_TWITTER_ACCESSTOKEN, GHD_TWITTER_ACCESSTOKENSECRET)

# Create API object 
api = tweepy.API(auth)
api

## reply message generator
also found in greenhouse summon

In [4]:
def TWEET_IMAGE_SAVER_AND_SELECTOR(code, max_count = 4, is_global=False):
    
    # Setup requests to find images in repository
    user = "dquintani"
    repo = "GreenhouseData"
    url = f"https://api.github.com/repos/{user}/{repo}/git/trees/master?recursive=1"
    r = requests.get(url)
    res = r.json()
    list_dir = []
    for file in res["tree"]:
        list_dir.append(file["path"])    
    
    # save randomly
    if is_global==True:
        path_dir = "global/"
        list_figs = [i for i in list_dir if i.startswith(path_dir)]
        try:
            selected_figs_paths = random.sample(list_figs, max_count)
        except:
            selected_figs_paths = []
    
    elif is_global == False: #ie is a country code
        path_dir = f"country_data/{code}_{STANDARD_COUNTRY_DICT[code]}/figures/"
        list_figs = [i for i in list_dir if i.startswith(path_dir)]
        
        selected_figs_paths = []
        # selected_figs_paths = random.sample(list_figs, max_count-1)
        selected_figs_paths.append(path_dir + f"{code}_relative_totals.png")
        selected_figs_paths.append(path_dir + f"{code}_GCP_Country_Highlight.png")
        selected_figs_paths.append(path_dir + f"{code}_GCP_1.png")
        selected_figs_paths.append(path_dir + f"{code}_Minx_top20_subsectors.png")
        display(selected_figs_paths)
        

    #DOWNLOAD AND SAVE THEM
    for i,j in enumerate(selected_figs_paths):
        image_url = f"https://dquintani.github.io/GreenhouseData/{j}"
        img_data = requests.get(image_url).content
        with open(f'image{i+1}.jpg', 'wb') as handler:
            handler.write(img_data)
        
    
    media_ids = []
    for i in range(max_count):
        try:
            res = api.media_upload(f"image{i+1}.jpg")
            media_ids.append(res.media_id)
        except:
            pass
    print(f" > saved and selected {len(media_ids)} images")
    return media_ids
    
# ---------------------------------------------------------------

# TWEET_IMAGE_SAVER_AND_SELECTOR("BES", 4)

# Tweet profile of the day
Currently, this code gets the latest tweet in the timeline, asks itself if a certain PERIOD of time has passed, and if enough time has passed, chooses a random country to profile.

In [5]:
def TWEET_COUNTRY_PROFILE_OF_THE_DAY():
    

    # get random country
    code = random.choice(list(STANDARD_COUNTRY_DICT.keys()))
    print("random country profile:", STANDARD_COUNTRY_DICT[code])

    # save images
    media_ids = TWEET_IMAGE_SAVER_AND_SELECTOR(code, 4)
    
    # write message
    MESSAGE = f"Greenhouse Data's country profile of the day: {STANDARD_COUNTRY_DICT[code]}\n\nMore greenhouse gas emission data and figures of this country: https://dquintani.github.io/GreenhouseData/country_data/{code}_{STANDARD_COUNTRY_DICT[code]}/"
    print(MESSAGE)
    
    # RUN
    api.update_status(status = MESSAGE, 
                    media_ids=media_ids)

    # print when success
    print("country profile tweet SUCCESS!")


    

TWEET_COUNTRY_PROFILE_OF_THE_DAY()

random country profile: Gambia


['country_data/GMB_Gambia/figures/GMB_relative_totals.png',
 'country_data/GMB_Gambia/figures/GMB_GCP_Country_Highlight.png',
 'country_data/GMB_Gambia/figures/GMB_GCP_1.png',
 'country_data/GMB_Gambia/figures/GMB_Minx_top20_subsectors.png']

 > saved and selected 4 images
Greenhouse Data's country profile of the day: Gambia

More greenhouse gas emission data and figures of this country: https://dquintani.github.io/GreenhouseData/country_data/GMB_Gambia/
country profile tweet SUCCESS!
